In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import os
from dotenv import load_dotenv
import requests
# Cargar variables de entorno
load_dotenv()

# Credenciales
SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')
print(f"Client ID: {SPOTIPY_CLIENT_ID}, Client Secret: {SPOTIPY_CLIENT_SECRET}, LastFM API Key: {LASTFM_API_KEY}")  


Client ID: 3e1baa583f3a480087dcc95d5ec7d197, Client Secret: c0e068b3e5da4447b32c04a9eea9f03d, LastFM API Key: 278ffdb96661827dbcda179b4aa59cfc


In [ ]:
 

# Parámetros
GENRES = ['jazz']
YEARS = [2010, 2024]

query_terms = [
    'jazz',
    'smooth jazz',
    'jazz fusion',
    'latin jazz',
    'contemporary jazz'
]

# Inicializar Spotify
try:
    sp = spotipy.Spotify(
        auth_manager=SpotifyClientCredentials(
            client_id=SPOTIPY_CLIENT_ID,
            client_secret=SPOTIPY_CLIENT_SECRET
        )
    )
    print("✅ Conexión a Spotify exitosa.")
except Exception as e:
    print(f"❌ Error de conexión a Spotify: {e}")


✅ Conexión a Spotify exitosa.


In [3]:
print(type(sp)) # Verificar el tipo de objeto sp

<class 'spotipy.client.Spotify'>


In [4]:
 #Prueba 2 Extraccion Spotipy
# --- DEFINICIÓN DE LA FUNCIÓN CON DOBLE BUCLE (MÁXIMA ROBUSTEZ Y LÍMITES) ---

def extract_spotify_data_jazz_ampliada(sp, years, query_terms): 
    """
    VERSION FINAL LIMITADA: Itera primero por AÑO y luego por CADA TÉRMINO de género.
    Máximo total: 500 canciones únicas. Máximo por año: 250 canciones únicas.
    """
    
    spotify_data = []
    MAX_TOTAL_SONGS = 500 # Nuevo límite global
    MAX_SONGS_PER_YEAR = 250 # Nuevo límite por año
    
    for year in years:
        
        # 1. VERIFICACIÓN DEL LÍMITE GLOBAL ANTES DE EMPEZAR EL AÑO
        if len(spotify_data) >= MAX_TOTAL_SONGS:
            print(f"🛑 LÍMITE GLOBAL ALCANZADO ({MAX_TOTAL_SONGS}). Saltando el año {year}.")
            break
            
        year_range = f'{year}-{year}'
        print(f"\n--- INICIANDO EXTRACCIÓN PARA EL AÑO {year} ---")
        
        songs_added_in_year = 0 # Contador para este año
        
        # BUENO 1: Iteramos sobre cada término de búsqueda 
        for term in query_terms:
            
            # 2. VERIFICACIÓN DEL LÍMITE ANUAL
            if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                print(f"🛑 LÍMITE ANUAL ALCANZADO ({MAX_SONGS_PER_YEAR}). Terminando la búsqueda para el año {year}.")
                break
            
            query = f'{term} year:{year_range}'
            print(f"   -> Buscando con término: {term}")
            
            try:
                # La API siempre trae un máximo de 50 por llamada
                results = sp.search(q=query, type='track', limit=50) 
                
                total_resultados_api = results['tracks']['total'] if results and 'tracks' in results else 0
                print(f"   -> Spotify reporta {total_resultados_api} resultados para '{term}'.")
                
                songs_added_in_term = 0
                
                # BUENO 2: Bucle de Paginación
                while results: # Mientras haya más resultados (no ponemos un límite aquí para exprimir los 1000)
                    
                    for track in results['tracks']['items']:
                        
                        track_id = track['id']
                        
                        # 3. VERIFICACIÓN DE LÍMITES DENTRO DEL BUCLE INTERNO
                        
                        # A. Límite Global alcanzado
                        if len(spotify_data) >= MAX_TOTAL_SONGS:
                            print(f"🛑 LÍMITE GLOBAL ({MAX_TOTAL_SONGS}) alcanzado. Terminando.")
                            return pd.DataFrame(spotify_data) # Terminamos inmediatamente
                            
                        # B. Límite Anual alcanzado
                        if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                            break # Salta al siguiente término de búsqueda
                        
                        # C. Deduplicación
                        # Usamos una forma más eficiente de comprobar duplicados si el volumen es grande
                        if any(d.get('ID_Spotify') == track_id for d in spotify_data):
                            continue
                            
                        # 4. Añadir la canción
                        data_row = {
                            'Artista': track['artists'][0]['name'],
                            'Género musical': f'jazz (Origen: {term})',
                            'Tipo': 'Canción',
                            'Nombre': track['name'],
                            'Año de lanzamiento': year,
                            'ID_Spotify': track_id,
                            'ID_Album': track['album']['id']
                        }
                        spotify_data.append(data_row)
                        songs_added_in_term += 1
                        songs_added_in_year += 1 # Contar para el límite anual

                    # Paginación
                    if results['tracks']['next']:
                        results = sp.next(results['tracks'])
                    else:
                        results = None
                    
                    # Aumentamos la pausa para evitar el Rate Limiting
                    time.sleep(1.5) 
                
                print(f"      -> Añadidas {songs_added_in_term} canciones ÚNICAS de '{term}'.")
                
            except Exception as e:
                print(f"   -> Error al buscar con término '{term}': {e}")
                time.sleep(3) # Pausa más larga después de un error para recuperarse
            
    return pd.DataFrame(spotify_data)

In [5]:
if sp:
    # Eliminamos el argumento 'max_total_songs=500'
    df_spotify_raw = extract_spotify_data_jazz_ampliada(sp, YEARS, query_terms) 
    
    print("\n--- RESULTADO FINAL DE jazz (ESTRATEGIA ROBUSTA) ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)}") 
    print(df_spotify_raw.head(10))


--- INICIANDO EXTRACCIÓN PARA EL AÑO 2010 ---
   -> Buscando con término: jazz
   -> Spotify reporta 49 resultados para 'jazz'.
      -> Añadidas 49 canciones ÚNICAS de 'jazz'.
   -> Buscando con término: smooth jazz
   -> Spotify reporta 100 resultados para 'smooth jazz'.
      -> Añadidas 97 canciones ÚNICAS de 'smooth jazz'.
   -> Buscando con término: jazz fusion
   -> Spotify reporta 100 resultados para 'jazz fusion'.
      -> Añadidas 97 canciones ÚNICAS de 'jazz fusion'.
   -> Buscando con término: latin jazz
   -> Spotify reporta 100 resultados para 'latin jazz'.
      -> Añadidas 7 canciones ÚNICAS de 'latin jazz'.
🛑 LÍMITE ANUAL ALCANZADO (250). Terminando la búsqueda para el año 2010.

--- INICIANDO EXTRACCIÓN PARA EL AÑO 2024 ---
   -> Buscando con término: jazz
   -> Spotify reporta 43 resultados para 'jazz'.
      -> Añadidas 43 canciones ÚNICAS de 'jazz'.
   -> Buscando con término: smooth jazz
   -> Spotify reporta 100 resultados para 'smooth jazz'.
      -> Añadidas 9

In [6]:
print(f"Número de canciones encontradas en df_spotify_raw: {len(df_spotify_raw)}")
print(f"Primeras filas (head) de df_spotify_raw:\n{df_spotify_raw.head()}")

Número de canciones encontradas en df_spotify_raw: 500
Primeras filas (head) de df_spotify_raw:
                   Artista       Género musical     Tipo               Nombre  \
0                  Quantic  jazz (Origen: jazz)  Canción  Cumbia Sobre El Mar   
1                Bop Alloy  jazz (Origen: jazz)  Canción            Jazzmatic   
2  Alessandro Alessandroni  jazz (Origen: jazz)  Canción        Jazz Whistler   
3             Caro Emerald  jazz (Origen: jazz)  Canción    A Night Like This   
4                     Sade  jazz (Origen: jazz)  Canción      Soldier of Love   

   Año de lanzamiento              ID_Spotify                ID_Album  
0                2010  61muv9lHtgN18qbm9oK9pH  0NiH3ROfMf9ZGYj07RP0B0  
1                2010  1cDM4wL3FKPO4JpK3YNiWT  6X8ohJVWti6xDto1k9mFlO  
2                2010  2eO48qUibU8IAM4tR51mrV  5LKhJ089woBuOf9e3eRzHC  
3                2010  7nuBU1HCcOwG5f1orN4ByW  7KYgUgJTrsxT4gkHRALi3R  
4                2010  0bgsj8kz5zWbQ6njk2Oqoz  3QzXOFN7Wh

In [14]:
# --- CELDA COMPLETA: EXTRACCIÓN Y UNIFICACIÓN DE ÁLBUMES ---

# Asegúrate de que las librerías necesarias (pandas, time) y la variable 'sp' estén disponibles
import pandas as pd
import time
# Nota: La variable 'sp' debe estar definida por la Celda 1.

# -------------------------------------------------------------------
# BLOQUE 1: DEFINICIÓN DE LA FUNCIÓN
# -------------------------------------------------------------------

def extract_album_details(sp, album_ids):
    """
    Extrae los metadatos completos (nombre, año, artista) de los álbumes.
    """
    album_details = []
    
    # El set ya se crea cuando se llama a la función con album_ids_list.
    unique_album_ids = album_ids
    
    print(f"\n--- INICIANDO EXTRACCIÓN DE METADATOS DE {len(unique_album_ids)} ÁLBUMES ---")
    
    for album_id in unique_album_ids:
        try:
            album_info = sp.album(album_id) 
            
            data_row = {
                'ID_Album': album_id,
                'Nombre_Album': album_info['name'],
                'Año_Lanzamiento_Album': album_info['release_date'][:4], # Tomamos solo el año
                'Artista_Principal_Album': album_info['artists'][0]['name'],
                'Tipo_Lanzamiento': album_info['album_type'] 
            }
            album_details.append(data_row)
            time.sleep(0.2) # Pausa para no saturar la API
            
        except Exception as e:
            # Capturamos errores como álbumes que han sido eliminados de Spotify
            print(f"   -> Error al extraer detalles del álbum {album_id}: {e}")
            pass
            
    print(f"✅ Extracción de {len(album_details)} álbumes completada.")
    return pd.DataFrame(album_details)

# -------------------------------------------------------------------
# BLOQUE 2: EJECUCIÓN Y FUSIÓN
# -------------------------------------------------------------------

# Verificación de que el DataFrame de canciones está disponible
if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    print("\n--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---")
    
    # 1. VERIFICACIÓN CRÍTICA: La columna 'ID_Album' debe existir.
    if 'ID_Album' not in df_spotify_raw.columns:
        print("❌ ERROR CRÍTICO: La columna 'ID_Album' NO se encontró. La Celda 3 falló al guardar el ID del álbum.")
    else:
        
        # 2. Obtener IDs únicos válidos de los álbumes
        album_ids_list = df_spotify_raw['ID_Album'].dropna().unique() 
        
        if len(album_ids_list) == 0:
            print("❌ LISTA VACÍA: Se encontraron canciones, pero 0 IDs de álbumes válidos. No se puede continuar.")
        else:
            print(f"✅ Se encontraron {len(album_ids_list)} IDs de álbumes únicos. Extrayendo ahora...")
            
            # 3. Ejecutar la función de extracción (Tardará unos minutos)
            df_album_details = extract_album_details(sp, album_ids_list)
            
            # 4. Fusionar (JOIN) los detalles del álbum con los datos de las canciones
            df_spotify_final = pd.merge(
                df_spotify_raw,
                df_album_details[['ID_Album', 'Nombre_Album', 'Año_Lanzamiento_Album', 'Artista_Principal_Album']],
                on='ID_Album',
                how='left' 
            )
            
            print(f"\n--- PROCESO DE ÁLBUMES COMPLETADO ---")
            print(f"DataFrame intermedio 'df_spotify_final' creado con {len(df_spotify_final)} filas.")
            print(df_spotify_final.head())
            
else:
    print("❌ ERROR: El DataFrame de canciones (df_spotify_raw) no está disponible. Vuelve a ejecutar la Celda 3.")


--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---
✅ Se encontraron 298 IDs de álbumes únicos. Extrayendo ahora...

--- INICIANDO EXTRACCIÓN DE METADATOS DE 298 ÁLBUMES ---
✅ Extracción de 298 álbumes completada.

--- PROCESO DE ÁLBUMES COMPLETADO ---
DataFrame intermedio 'df_spotify_final' creado con 500 filas.
                   Artista            Género musical     Tipo  \
0                  Quantic  Reggaeton (Origen: jazz)  Canción   
1                Bop Alloy  Reggaeton (Origen: jazz)  Canción   
2  Alessandro Alessandroni  Reggaeton (Origen: jazz)  Canción   
3             Caro Emerald  Reggaeton (Origen: jazz)  Canción   
4                     Sade  Reggaeton (Origen: jazz)  Canción   

                Nombre  Año de lanzamiento              ID_Spotify  \
0  Cumbia Sobre El Mar                2010  61muv9lHtgN18qbm9oK9pH   
1            Jazzmatic                2010  1cDM4wL3FKPO4JpK3YNiWT   
2        Jazz Whistler                2010  2eO48qUibU8IAM4tR51mrV   
3    A Night 

In [15]:
def extract_lastfm_artist_details(artist_name):
    """Extrae biografía y estadísticas de popularidad de un artista."""
    
    url = 'http://ws.audioscrobbler.com/2.0/'
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': LASTFM_API_KEY,
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'artist' in data:
            artist_info = data['artist']
            
            # Extraer los datos relevantes (Biografía, Popularidad/Plays)
            biography = artist_info.get('bio', {}).get('summary', 'N/A')
            playcount = artist_info.get('stats', {}).get('playcount', '0')
            listeners = artist_info.get('stats', {}).get('listeners', '0')
            
            return {
                'Artista': artist_name,
                'Biografia_Resumen': biography,
                'Playcount_LastFM': playcount,
                'Listeners_LastFM': listeners,
            }
    except Exception:
        # En caso de error de conexión o artista no encontrado, devuelve None
        pass 
        
    return None

# --- Ejecución para Artistas Únicos ---

if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    # Obtener la lista única de artistas para evitar búsquedas repetidas
    unique_artists = df_spotify_raw['Artista'].unique()
    lastfm_results = []
    
    print("\n--- EXTRACCIÓN DETALLES LAST.FM ---")
    
    for artist in unique_artists: 
        details = extract_lastfm_artist_details(artist)
        if details:
            lastfm_results.append(details)
        time.sleep(0.5) # Pausa crucial para no sobrecargar la API

    df_lastfm = pd.DataFrame(lastfm_results)
    
    # 5. UNIFICACIÓN DE DATOS
    # Fusionar los datos de Spotify con los detalles de Last.fm
    df_final_jazz   = pd.merge(
        df_spotify_raw, 
        df_lastfm[['Artista', 'Playcount_LastFM', 'Listeners_LastFM']],
        on='Artista', 
        how='left' # Usamos left join para mantener todas las canciones de Spotify
    )
    
    print("\n--- DATAFRAME FINAL DE JAZZ LISTO PARA ALMACENAMIENTO ---")
    print(df_final_jazz.head())
    print(f"Total de registros finales para jazz: {len(df_final_jazz)}")


--- EXTRACCIÓN DETALLES LAST.FM ---

--- DATAFRAME FINAL DE JAZZ LISTO PARA ALMACENAMIENTO ---
                   Artista            Género musical     Tipo  \
0                  Quantic  Reggaeton (Origen: jazz)  Canción   
1                Bop Alloy  Reggaeton (Origen: jazz)  Canción   
2  Alessandro Alessandroni  Reggaeton (Origen: jazz)  Canción   
3             Caro Emerald  Reggaeton (Origen: jazz)  Canción   
4                     Sade  Reggaeton (Origen: jazz)  Canción   

                Nombre  Año de lanzamiento              ID_Spotify  \
0  Cumbia Sobre El Mar                2010  61muv9lHtgN18qbm9oK9pH   
1            Jazzmatic                2010  1cDM4wL3FKPO4JpK3YNiWT   
2        Jazz Whistler                2010  2eO48qUibU8IAM4tR51mrV   
3    A Night Like This                2010  7nuBU1HCcOwG5f1orN4ByW   
4      Soldier of Love                2010  0bgsj8kz5zWbQ6njk2Oqoz   

                 ID_Album  Popularidad Playcount_LastFM Listeners_LastFM  
0  0NiH3ROfMf9ZGY

In [20]:
# --- CELDA 6: FUSIÓN DE SPOTIFY (CARGADO DE CSV) + LAST.FM Y GUARDADO FINAL ---

import pandas as pd
import os

print("\n--- INICIANDO FUSIÓN DE DATOS (SPOTIFY + LAST.FM) ---")

# 1. CARGA DEL CSV INICIAL DE SPOTIFY
# CRÍTICO: Debe ser el nombre del archivo guardado en la Celda 3 (o la Celda 4, si la ejecutaste).
spotify_input_filename = 'jazz_final.csv' 
df_spotify_cargado = None # Inicializamos la variable

if os.path.exists(spotify_input_filename):
    df_spotify_cargado = pd.read_csv(spotify_input_filename)
    print(f"✅ DataFrame de Spotify cargado desde CSV: {len(df_spotify_cargado)} filas.")
else:
    print(f"❌ ERROR: No se encuentra el archivo CSV inicial de Spotify: {spotify_input_filename}")


if df_spotify_cargado is not None and 'df_lastfm' in locals():
    
    # 2. Realizamos la fusión (Merge) usando df_spotify_cargado como base
    df_proyecto_final = pd.merge(
        df_spotify_cargado, 
        df_lastfm,
        on='Artista', 
        how='left'
    )
    
    # 3. Limpieza de las columnas de Last.fm
   
    
    # Rellenar valores nulos con 0 y asegurar que sean enteros
    for col in ['Playcount_LastFM', 'Listeners_LastFM']:
        if col in df_proyecto_final.columns:
            # Primero rellenamos NaN con 0.0 (float) y luego convertimos a int
            df_proyecto_final[col] = df_proyecto_final[col].fillna(0).astype(int)
    
    
    # 4. Guardar el archivo CSV FINAL
    output_filename = 'data_jazz_final2.csv'
    
    df_proyecto_final.to_csv(
        output_filename, 
        index=False, 
        encoding='utf-8'
    )
    
    print(f"\n🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉")
    print(f"Total de registros finales: {len(df_proyecto_final)}")
    print("--- Primeras 5 filas con datos clave ---")
  
    print(f"✅ Archivo final guardado: {output_filename}")
    
else:
    print("❌ ERROR DE FUSIÓN: Asegúrate de que la Celda 5 (Last.fm) se haya ejecutado exitosamente para crear 'df_lastfm'.")


--- INICIANDO FUSIÓN DE DATOS (SPOTIFY + LAST.FM) ---
✅ DataFrame de Spotify cargado desde CSV: 500 filas.

🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉
Total de registros finales: 500
--- Primeras 5 filas con datos clave ---
✅ Archivo final guardado: data_jazz_final2.csv
